# Launch post-run analysis jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [2]:
from ipywidgets import *

In [3]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/run_scripts


In [4]:
val_files={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
          '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
}

In [5]:
img_size=128
results_loc='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size)
print(results_loc)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/


In [6]:
# ls /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/

In [7]:
# ip_folder='20210409_083631_bsize128_with_bnorm_200kdata'

In [12]:
dir_lst=[i.split('/')[-1] for i in glob.glob(results_loc+'20210*')]
w=interactive(lambda x: x, x=ToggleButtons(options=dir_lst))
display(w)

interactive(children=(ToggleButtons(description='x', options=('20210422_132856_bsize128_bnorm_new_decay', '202…

In [13]:
result=w.result
ip_folder=results_loc+result
print(ip_folder)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210507_084712_bsize128_bnorm_new_decay


In [14]:
### Default dictionary
dict_pars={'job_name':'analysis', 'queue':'debug',
           'bins_type':'uneven',# even on uneven
           'time':'00:30:00'}

dict_pars['val_file']=val_files[str(img_size)]
dict_pars['ip_folder']=ip_folder

print(dict_pars)

{'job_name': 'analysis', 'queue': 'debug', 'bins_type': 'uneven', 'time': '00:30:00', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'ip_folder': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210507_084712_bsize128_bnorm_new_decay'}


In [15]:
assert os.path.isdir(dict_pars['ip_folder'])

In [16]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/'

python $code_dir/3a_analysis_pandas.py -f {ip_folder}  -v {val_file} --bins_type {bins_type}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [17]:
# bash_strg

In [18]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
fname='batch_analysis.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_analysis.sh


In [19]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [20]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:30:00
#SBATCH --job-name=analysis

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

conda activate v3
code_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/'

python $code_dir/3a_analysis_pandas.py -f /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210507_084712_bsize128_bnorm_new_decay  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy --bins_type uneven
conda deactivate
echo "--end date" `date` `date +%s`


### Submit jobs to cori queue

In [21]:
%%bash -s "$filename" ## Use python variable in bash
sbatch $1

Submitted batch job 42443214
